In [ ]:
import argparse
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
# from text_loader import TextLoader

from tensorflow.contrib.layers import batch_norm

from sklearn import metrics
print(tf.__version__)

In [5]:
data=pd.read_csv('celeba-dataset/newdataset.csv')

In [6]:
total=202599*25
total

5064975

In [7]:
# print(data.sum())
zero=0
one=0
for col in data.columns[1:-1]:
    zero=zero+data[col].value_counts()[0]
    one=one+data[col].value_counts()[1]
print(zero)
print(one)
print(zero+one)
print(zero/total)
data

4315799
749176
5064975
0.8520869303402288


,image_id,has bags under eyes,is bald,has bangs,has black hair,has blond hair,has brown hair,has bushy eyebrows,is chubby,is wearing eyeglasses,...,has no beard,has oval face,has pale skin,has a pointy nose,has sideburns,is smiling,is wearing a hat,is young,the female,sentence
0,000001.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,1,A opening her mouth slightly woman with no bea...
1,000002.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,A opening her mouth slightly woman with high c...
2,000003.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,A young man with a pointy nose has narrow eyes...
3,000004.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,A woman has a pointy nose and no beard
4,000005.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,A woman is young
5,000006.jpg,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,1,A opening her mouth slightly woman with brown ...
6,000007.jpg,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,A man with bushy eyebrows and black hair has b...
7,000008.jpg,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,A young man with bags under eyes has black hai...
8,000009.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,A young woman with high cheekbones and bangs i...
9,000010.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,A young woman has high cheekbones and no beard


In [8]:
for col in data.columns[1:-1]:
    zero=data[col].value_counts()[0]
    one=data[col].value_counts()[1]
    print(col,"{}".format(zero/(zero+one)))
    

has bags under eyes 0.89600639687264
is bald 0.9882724001599218
has bangs 0.9235386156891199
has black hair 0.8744761820147188
has blond hair 0.9236866914446764
has brown hair 0.8914456636014986
has bushy eyebrows 0.9278574918928524
is chubby 0.9716928513961076
is wearing eyeglasses 0.9634302242360525
has a goatee 0.9668409024723715
has gray hair 0.9768310801139196
has high cheekbones 0.7829752367978124
the male 0.5832457218446291
is opening mouth slightly 0.7628171906080484
has a mustache 0.9781489543383729
has narrow eyes 0.9431191664322134
has no beard 0.5432998188539924
has oval face 0.8606113554361078
has pale skin 0.9767669139531785
has a pointy nose 0.8577683009294221
has sideburns 0.9701380559627639
is smiling 0.7720916687644065
is wearing a hat 0.9725516907783355
is young 0.5778064057571853
the female 0.41675427815537097


In [9]:
x=data['sentence']
y=np.array(data.iloc[:,1:26]).tolist()
# y=list(data.iloc[0:,1:2])
# y = [1 if x == '1' else 0 for x in data['is chubby'] ]
y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [10]:
FLAGS = None
MAX_DOCUMENT_LENGTH = 20 # 한 문장의 단어 갯수

EMBEDDING_SIZE = 150 # 임베딩 차원수
n_words = 0 # 등장하는 단어 종류(이후에 계산됨)
MAX_LABEL = 25 # 특징 갯수
WORDS_FEATURE = 'words'
learning_rate = 0.0001

sess = tf.Session()


In [11]:
def estimator_spec_for_softmax_classification (logits, labels, mode):

    """Returns EstimatorSpec instance for softmax classification."""

#     predicted = tf.argmax(logits, 1)
#     predicted = tf.round(tf.nn.softmax(logits))
#     predicted = tf.sigmoid(logits)
    logits = tf.identity(logits, name='logits')
    prob = tf.nn.sigmoid(logits , name='prob')
    predicted_classes = tf.round(prob, name='predicted_classes')
    
    print("666")
    print(logits)
#     print(type(tf.Session().run(predicted)))

    # case 1) inference mode
    
    # ModeKey가 PREDICT이면 inference mode이다. inference는 학습시킨 모델을 새로운 데이터에 적용시키는 것을 말한다.
    if mode == tf.estimator.ModeKeys.PREDICT: 
        
        print("ppp")
        return tf.estimator.EstimatorSpec(mode=mode, predictions={
            'logits': logits,
            'prob': prob,
            'predicted_classes': predicted_classes,
        })


    # case 2) training mode

#     onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)


    print("777")
    print(type(logits))

    loss = tf.losses.sigmoid_cross_entropy(labels, logits=prob)
    print("loss")
    print(loss)
#     print('loss: {0:f}'.format(loss))
    if mode == tf.estimator.ModeKeys.TRAIN:
        
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.cast(labels,tf.float32),predicted_classes), tf.float32), name='acc')
        acc_row = tf.reduce_mean(tf.cast(tf.reduce_all(tf.equal(tf.cast(labels,tf.float32), predicted_classes), axis=1), tf.float32), name='acc_row')
    
        labels = tf.identity(labels , name='labels')

        
        print("ttt")
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op, predictions=logits)

    # case 3) evaluation mode
    
    print("eee!")
    print(labels)
    print(predicted_classes)
    
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)}
    print("eee?")
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [12]:
def rnn_model(features, labels, mode):

    """RNN model to predict from sequence of words to a class."""
    
    # 단어들의 index를 embeddings로 바꾼다. 
    # [n_words(단어 개수), EMBEDDING_SIZE]의 크기를 가지는 matrix를 만들고 순서를 나타내는 단어들의 index를 
    # [batch_size, sequence_length, EMBEDDING_SIZE]에 mapping 한다.
    # word_vectors와 word_list를 만든다.
    word_vectors = tf.contrib.layers.embed_sequence(features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)
    word_list = tf.unstack(word_vectors, axis=1)

    
    # embedding size와 같은 hidden size를 가지는 GUR cell을 만든다.
#     cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE / 2)

    
    # MAX_DOCUMENT_LENGTH의 길이를 가지는 RNN을 만든다. 그리고 각 유닛에 input으로 word_list를 준다.
    # (output, state)쌍이 return된다.
    print("@@@")
#     out1, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)
    
#     resout1 = [w + o for w, o in zip(out1, word_list)]
    
#     out2, encoding = tf.contrib.rnn.static_rnn(cell, resout1, dtype=tf.float32)
    
#     resout2 = [w + o for w, o in zip(out2, resout1)]
    
#     out3, encoding = tf.contrib.rnn.static_rnn(cell, resout2, dtype=tf.float32)
    
    
    out1, state1, state2 = tf.contrib.rnn.static_bidirectional_rnn(cell, cell, word_list, dtype=tf.float32)
    
    resout1 = [w + o for w, o in zip(out1, word_list)]
    
    out2, state1, state2 = tf.contrib.rnn.static_bidirectional_rnn(cell, cell, resout1, dtype=tf.float32)
    
    resout2 = [w + o for w, o in zip(out2, resout1)]
    
    out3, state1, state2 = tf.contrib.rnn.static_bidirectional_rnn(cell, cell, resout2, dtype=tf.float32)
    
    print("###")
#     print(out2)
#     print("###")
#     print(state1)
#     print("###")
#     print(state2)
    print("###")
#     print(encoding)
    print(tf.concat([state1,state2],1))
    print("###")
    ##########중요 여기바꿔야
    # 마지막 유닛의 state 값을 softmax classification의 feature로 넘겨준다.
#     logits = tf.layers.dense(encoding, MAX_LABEL, activation=tf.nn.relu)
    logits = tf.layers.dense(tf.concat([state1,state2],1), MAX_LABEL, activation=tf.nn.relu)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        print("trainbatch")
        logits = tf.contrib.layers.batch_norm(logits,is_training=True)
    else:
        print("testbatch")
        logits = tf.contrib.layers.batch_norm(logits,is_training=False)
        
        
          
          
    return estimator_spec_for_softmax_classification(logits=logits, labels=labels, mode=mode)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
def main(unused_argv):
    global n_words
    global vocab_processor
    global classifier
    
    # train data set, test data set을 나눠준다.
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
    # 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    
    x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
    x_transform_test = vocab_processor.transform(x_test)
    x_train = np.array(list(x_transform_train))
    x_test = np.array(list(x_transform_test))
    
    
    n_words = len(vocab_processor.vocabulary_)
    print('Total words : %d', n_words)
    
    # 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
    model_fn = rnn_model
    classifier = tf.estimator.Estimator(model_fn=model_fn)
    
    
    # Train
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_train},
        y = y_train,
        batch_size = 256,
        num_epochs = 20,
        shuffle = True) # shuffle = True : 
    
    
    
    tensors_to_log = {'prob': 'prob', 'labels': 'labels', 'predicted_classes': 'predicted_classes', 'acc': 'acc', 'acc_row': 'acc_row','logits': 'logits'}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

    
    classifier.train(input_fn = train_input_fn, hooks=[logging_hook])

    # Predict
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_test},
        y = y_test,
        num_epochs = 1,
        shuffle = False)
    predictions = classifier.predict(input_fn = test_input_fn)
    y_predicted = np.array(list(p['prob'] for p in predictions))
    
    
    # Score using tensorflow
    score = classifier.evaluate(input_fn = test_input_fn)
    print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
    print()

In [15]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--test_with_fake_data',
        default = False,
        help = 'Test the example code with fake data',
        action = 'store_true')
    parser.add_argument(
        '--bow model',
        default = False,
        help = 'Run with BOW model instead of RNN',
        action = 'store_true')
    FLAGS, unparsed = parser.parse_known_args()
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run(main=main, argv = [sys.argv[0]] + unparsed)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Total words : %d 48
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpp_7nmm1l', '_save_checkpoints_steps': None, '_device_fn': None, '_save_summary_steps': 100, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f520c9006a0>, '_session_config': None, '_service': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_evaluation_master': '', '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0}
INFO:tensorflow:Calling model_fn.
@@@
###
###
Tensor("concat_60:0", shape=(?, 150), dty

INFO:tensorflow:loss = 0.8520289, step = 400 (3.561 sec)
INFO:tensorflow:global_step/sec: 27.8346
INFO:tensorflow:acc = 0.9117187, acc_row = 0.11328125, labels = [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]], logits = [[-0.38412708 -0.2909021  -0.3486532  ...  0.56700057  0.895839
  -1.3030441 ]
 [-0.38412708 -0.2909021  -0.3486532  ...  0.49389714  1.0574023
  -1.2424117 ]
 [-0.38412708 -0.2909021  -0.3486532  ... -0.31295815  0.7053568
   0.79295784]
 ...
 [-0.38412708 -0.2909021  -0.3486532  ... -0.31295815  1.1311361
   0.6800035 ]
 [-0.38412708 -0.2909021  -0.3486532  ... -0.31295815  0.8144871
   0.65970856]
 [-0.38412708 -0.2909021  -0.3486532  ... -0.31295815 -1.0101833
   0.9182414 ]], predicted_classes = [[0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]], prob = [[0.40513188 0.42778304 0.41370904 ...

INFO:tensorflow:loss = 0.83620393, step = 1000 (3.612 sec)
INFO:tensorflow:global_step/sec: 27.6824
INFO:tensorflow:acc = 0.951875, acc_row = 0.26171875, labels = [[0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]], logits = [[-0.42256233 -0.38841125 -0.43235934 ... -0.35476148  1.1774629
   0.7827637 ]
 [-0.42256233 -0.38841125 -0.43235934 ... -0.35476148 -0.92022383
   0.8206483 ]
 [-0.42256233 -0.38841125 -0.10381985 ... -0.35476148 -0.7419352
   0.81027836]
 ...
 [-0.42256233 -0.38841125 -0.43235934 ... -0.35476148 -0.92022383
   0.8291425 ]
 [-0.42256233 -0.38841125 -0.43235934 ... -0.35476148 -0.92022383
   0.82725525]
 [-0.42256233 -0.38841125 -0.43235934 ... -0.35476148 -0.92022383
   0.7769453 ]], predicted_classes = [[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]], prob = [[0.39590374 0.40409982 0.393563

INFO:tensorflow:loss = 0.8357204, step = 1600 (3.637 sec)
INFO:tensorflow:global_step/sec: 27.5376
INFO:tensorflow:acc = 0.96125, acc_row = 0.30859375, labels = [[0 1 0 ... 0 0 0]
 [0 0 1 ... 0 1 1]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]], logits = [[-0.5242212   3.231011   -0.4631642  ...  2.948254   -1.0386958
  -1.2027837 ]
 [-0.5242212  -0.34337288  3.2719572  ... -0.44051227  0.87774175
   0.75025976]
 [-0.5242212  -0.34337288 -0.4631642  ... -0.44051227  0.9854073
  -1.2027837 ]
 ...
 [-0.5242212  -0.32494855 -0.4631642  ... -0.44051227 -1.0386958
  -1.2027837 ]
 [-0.5242212  -0.34337288 -0.4631642  ... -0.44051227  0.98637843
   0.83249146]
 [-0.5242212  -0.34337288 -0.4631642  ... -0.44051227 -1.0386958
   0.82968163]], predicted_classes = [[0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]], prob = [[0.37186572 0.96198475 0.38623548 .

INFO:tensorflow:loss = 0.8242402, step = 2200 (3.613 sec)
INFO:tensorflow:global_step/sec: 27.9952
INFO:tensorflow:acc = 0.965625, acc_row = 0.3828125, labels = [[1 0 0 ... 0 0 1]
 [0 0 1 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 1 1]
 [0 0 0 ... 0 1 1]], logits = [[ 2.391882   -0.39658257 -0.566756   ... -0.5188727  -1.2847178
   0.94164395]
 [-0.5772325  -0.39658257  2.4512632  ... -0.5188727   0.652358
   0.90179807]
 [-0.5772325  -0.39658257 -0.566756   ... -0.5188727  -1.2847178
  -1.1547701 ]
 ...
 [-0.5772325  -0.39658257 -0.566756   ... -0.5188727  -1.2847178
  -1.1547701 ]
 [-0.5772325  -0.39658257  2.7422507  ... -0.5188727   0.6822201
   0.83395976]
 [-0.5772325  -0.39658257 -0.566756   ... -0.5188727   0.6413735
   0.93093765]], predicted_classes = [[1. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]], prob = [[0.9162062  0.40213367 0.36198568 ... 

...

INFO:tensorflow:loss = 0.74223787, step = 11200 (3.618 sec)
INFO:tensorflow:global_step/sec: 27.2425
INFO:tensorflow:acc = 0.99953127, acc_row = 0.98828125, labels = [[0 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]], logits = [[-1.4385403  -1.2202876   1.7357161  ... -1.2546858  -1.7492261
   0.8078143 ]
 [-1.4385403  -1.2202876  -1.4023019  ... -1.2546858  -1.7492261
   0.8492143 ]
 [-1.4385403  -1.2202876  -1.4023019  ... -1.2546858  -1.7492261
   0.81752497]
 ...
 [-1.4385403  -1.2202876  -1.4023019  ... -1.2546858  -1.7492261
   0.80528694]
 [-1.4385403  -1.2202876  -1.4023019  ... -1.2546858  -1.7492261
   0.85487205]
 [-1.4385403  -1.2202876  -1.4023019  ... -1.2546858   0.2128869
   0.8174926 ]], predicted_classes = [[0. 0. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]], prob = [[0.19177149 0.22788584 0.8501421

INFO:tensorflow:loss = 0.73825455, step = 11800 (3.652 sec)
INFO:tensorflow:global_step/sec: 27.3163
INFO:tensorflow:acc = 0.9996875, acc_row = 0.9921875, labels = [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 1]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], logits = [[-1.5404031  -1.2658178  -1.466759   ... -1.3111304  -1.6748245
   0.8043805 ]
 [-1.5404031  -1.2658178  -1.466759   ... -1.3111304  -1.6748245
   0.7794773 ]
 [-1.5404031  -1.2658178  -1.466759   ... -1.3111304  -1.6748245
  -1.2686273 ]
 ...
 [-1.5404031  -1.2658178  -1.466759   ... -1.3111304   0.40020952
   0.77353346]
 [ 1.3390831  -1.2658178  -1.466759   ... -1.3111304  -1.6748245
  -1.2686273 ]
 [-1.5404031  -1.2658178  -1.466759   ... -1.3111304  -1.6748245
  -1.2686273 ]], predicted_classes = [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], prob = [[0.17647669 0.21997404 0.18743573

INFO:tensorflow:loss = 0.7345221, step = 12400 (3.593 sec)
INFO:tensorflow:global_step/sec: 27.3607
INFO:tensorflow:acc = 1.0, acc_row = 1.0, labels = [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]], logits = [[-1.5131693  -1.2751153  -1.5145876  ... -1.3528631  -1.7532978
   0.75099474]
 [-1.5131693  -1.2751153  -1.5145876  ... -1.3528631   0.2998929
   0.7559771 ]
 [-1.5131693  -1.2751153  -1.5145876  ... -1.3528631  -1.7532978
  -1.3107774 ]
 ...
 [ 1.9797212  -1.2751153  -1.5145876  ... -1.3528631   0.22561061
  -1.3107774 ]
 [-1.5131693  -1.2751153  -1.5145876  ... -1.3528631  -1.7532978
  -1.3107774 ]
 [-1.5131693  -1.2751153  -1.5145876  ... -1.3528631   0.21018738
   0.7281045 ]], predicted_classes = [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]], prob = [[0.18046959 0.21838287 0.1802599  ... 0.20540

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
x_transform_result = vocab_processor.transform(x)
x_result = np.array(list(x_transform_result))
y_result = np.array(y)
    

    # Predict
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_result},
    y = y_result,
    num_epochs = 1,
    shuffle = False)

predictions = classifier.predict(input_fn = test_input_fn)
y_logits = np.array(list(p['logits'] for p in predictions))
print(y_logits)
print(y_logits.shape)

predictions = classifier.predict(input_fn = test_input_fn)
y_prob = np.array(list(p['prob'] for p in predictions))
print(y_prob)
print(y_prob.shape)

predictions = classifier.predict(input_fn = test_input_fn)
y_predicted_classes = np.array(list(p['predicted_classes'] for p in predictions))
print(y_predicted_classes)
print(y_predicted_classes.shape)
    
# Score using tensorflow
score = classifier.evaluate(input_fn = test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
print()

INFO:tensorflow:Calling model_fn.
@@@
###
###
Tensor("concat_60:0", shape=(?, 150), dtype=float32)
###
testbatch
666
Tensor("logits:0", shape=(?, 25), dtype=float32)
ppp
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp_7nmm1l/model.ckpt-12663
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[-1.2069418  -1.2286294  -1.2161264  ... -1.2273476   1.3720585
   3.2946615 ]
 [ 1.7686007  -1.2286294  -1.2161264  ... -1.2273476  -0.86196303
   3.266552  ]
 [-1.2069418  -1.2286294  -1.2161264  ... -1.2273476   1.3704385
  -0.01974662]
 ...
 [-1.2069418  -1.2286294  -1.2161264  ... -1.2273476  -0.8910116
  -0.01974662]
 [-1.2069418  -1.2286294  -1.2161264  ... -1.2273476   1.3385092
   3.3211372 ]
 [-1.2069418  -1.2286294  -1.2161264  ... -1.2273476   1.383912
   3.3858972 ]]
(202599, 25)
INFO:tensorflow:Calling model_fn.
@@@
###
###
Tensor("concat_60:0", shape=(?, 150), dtype=float3

In [17]:
attr=pd.DataFrame(y_prob)
# attr.to_csv('celeba-dataset/attr.csv',index=False)
attr

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.847598,0.231347,0.226592,0.231567,0.227144,0.818762,0.226646,0.797713,0.964245
1,0.854284,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.813673,0.226646,0.296929,0.963263
2,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.893830,0.857029,0.231347,0.226592,0.827042,0.227144,0.236010,0.226646,0.797451,0.495064
3,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.848516,0.231347,0.226592,0.828885,0.227144,0.236010,0.226646,0.290901,0.962808
4,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.792245,0.965147
5,0.230243,0.226421,0.228619,0.230902,0.229227,0.880379,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.844832,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.789205,0.964984
6,0.843310,0.226421,0.228619,0.856332,0.229227,0.230029,0.876754,0.227724,0.227509,0.227404,...,0.228122,0.838772,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.290901,0.495064
7,0.853705,0.226421,0.228619,0.851993,0.229227,0.230029,0.228527,0.227724,0.227509,0.231799,...,0.228122,0.326709,0.231347,0.226592,0.827412,0.227144,0.236010,0.226646,0.796232,0.495064
8,0.230243,0.226421,0.793946,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.794794,0.964835
9,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,0.227404,...,0.228122,0.845152,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.797354,0.963577


In [19]:
idnum=pd.DataFrame(list(data['image_id']),columns=['image_id'])
newdataset=pd.concat([idnum,attr],axis=1)
newdataset.to_csv('celeba-dataset/attr.csv',index=False)
newdataset

,image_id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,000001.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.847598,0.231347,0.226592,0.231567,0.227144,0.818762,0.226646,0.797713,0.964245
1,000002.jpg,0.854284,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.813673,0.226646,0.296929,0.963263
2,000003.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.893830,0.857029,0.231347,0.226592,0.827042,0.227144,0.236010,0.226646,0.797451,0.495064
3,000004.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.848516,0.231347,0.226592,0.828885,0.227144,0.236010,0.226646,0.290901,0.962808
4,000005.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.792245,0.965147
5,000006.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.880379,0.228527,0.227724,0.227509,...,0.228122,0.844832,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.789205,0.964984
6,000007.jpg,0.843310,0.226421,0.228619,0.856332,0.229227,0.230029,0.876754,0.227724,0.227509,...,0.228122,0.838772,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.290901,0.495064
7,000008.jpg,0.853705,0.226421,0.228619,0.851993,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.326709,0.231347,0.226592,0.827412,0.227144,0.236010,0.226646,0.796232,0.495064
8,000009.jpg,0.230243,0.226421,0.793946,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.326709,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.794794,0.964835
9,000010.jpg,0.230243,0.226421,0.228619,0.230902,0.229227,0.230029,0.228527,0.227724,0.227509,...,0.228122,0.845152,0.231347,0.226592,0.231567,0.227144,0.236010,0.226646,0.797354,0.963577


In [1]:

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)
# print(x_test)
# print(x_test.shape)
# print(y_test)
# vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    
# x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
# x_transform_test = vocab_processor.transform(x_test)
# x_train = np.array(list(x_transform_train))
# x_test = np.array(list(x_transform_test))
# print(x_test)
# print(x_test.shape)
# print(y_test)

# x_transform = vocab_processor.transform(x)
# x1 = np.array(list(x_transform))
# y1 = np.array(y)
# print(x1)
# print(x1.shape)
# print(y1)

In [ ]:
# global n_words
# global vocab_processor
# global classifier
    
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
# # 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
# vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    
# x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
# x_transform_test = vocab_processor.transform(x_test)
# x_train = np.array(list(x_transform_train))
# x_test = np.array(list(x_transform_test))
    
    
# n_words = len(vocab_processor.vocabulary_)
# print('Total words : %d', n_words)
    
# # 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
# model_fn = rnn_model
# classifier = tf.estimator.Estimator(model_fn=model_fn)
    
    
# # Train
# train_input_fn = tf.estimator.inputs.numpy_input_fn(
#     x = {WORDS_FEATURE : x_train},
#     y = y_train,
#     batch_size = 256,
#     num_epochs = 1,
#     shuffle = True) # shuffle = True : 
    
    
    
# tensors_to_log = {"prob": 'prob', 'labels': 'labels', 'predicted_classes': 'predicted_classes', 'acc': 'acc', 'acc_row': 'acc_row'}
# logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

    
# classifier.train(input_fn = train_input_fn, hooks=[logging_hook])

#     # Predict
# test_input_fn = tf.estimator.inputs.numpy_input_fn(
#     x = {WORDS_FEATURE : x_test},
#     y = y_test,
#     num_epochs = 1,
#     shuffle = False)
# predictions = classifier.predict(input_fn = test_input_fn)
# y_predicted = np.array(list(p['prob'] for p in predictions))
    
    
# # Score using tensorflow
# score = classifier.evaluate(input_fn = test_input_fn)
# print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
# print()

In [ ]:
vocab_processor.vocabulary_.reverse(47)

In [ ]:
classifier.predict()

In [ ]:

# train data set, test data set을 나눠준다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
print(x_train)
print(y_train)
print(x_test)
print(y_test)
# 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)

x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
x_transform_test = vocab_processor.transform(x_test)
x_train = np.array(list(x_transform_train))
x_test = np.array(list(x_transform_test))


In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_test},
    y = y_test,
    num_epochs = 1,
    shuffle = False)

# Score using tensorflow
score = classifier.evaluate(input_fn = test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
print(score)

In [ ]:
for p in predictions:
    print(p)